# Preguntas
- Cuánto pagan de expensas los departamentos en CABA según barrio?
  - La media de las expensas según barrio
  - Media del ratio entre alquiler y expensas
  - media de expensas por m2
  - Distribución de los alquileres en un mapa. Ideal: que muestre la información en un tooltip / hover de los aspectos más importantes.
  - Tipo de ajuste del precio: periodicidad e indexación. 

Para ello:
- Método para detectar outliers
- Limpieza de las direcciones.

In [32]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import re

from config.config_jupyter import set_wd
set_wd()
from src.zonaprop_wrangling import main_zonaprop_wrangling

c:\Users\mbasualdo\Documents\Scrap-alquileres-caba


In [33]:
df = pd.read_parquet("output\zonaprop_alquiler_departamentos_2024-04-09_12-47-43.pkt")
df = main_zonaprop_wrangling(df)

<>:1: SyntaxWarning:

invalid escape sequence '\z'

<>:1: SyntaxWarning:

invalid escape sequence '\z'

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_2956\2384882526.py:1: SyntaxWarning:

invalid escape sequence '\z'



In [34]:
df

,id,Price,Location,Address,Description,Expensas,Link,scrap_date,currency,metros_cuadrados,ambientes,dormitorios,baños,cocheras,Alquiler / m2
0,53380145,550000.0,"Palermo, Capital Federal",Bulnes al 1900,"Semipiso 3 ambientes, con balcón al frente. Mu...",65000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,53.0,3.0,2.0,NaN,NaN,10377.358491
1,52999788,4500.0,"Puerto Madero, Capital Federal",Julieta Lanteri al 1200,Alquiler de departamento 4 ambientes. Impecabl...,650000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,USD,230.0,4.0,3.0,3.0,2.0,19.565217
2,53244485,800.0,"Puerto Madero, Capital Federal",Azopardo 770,Alquiler temporarioquartier Madero Urbano. Mon...,107500.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,USD,37.0,1.0,NaN,NaN,NaN,21.621622
3,52990594,1800.0,"Belgrano, Capital Federal",3 de Febrero 1800,Excelente semi piso de 158 m2 en Torre de cate...,230000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,USD,157.0,4.0,3.0,2.0,2.0,11.464968
4,53183332,700.0,"Palermo Soho, Palermo",Scalabrini al 1200,*El alquiler es de usd 750 + mas gastos de exp...,75000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,USD,58.0,2.0,1.0,NaN,NaN,12.068966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9101,45618429,245000.0,"Centro / Microcentro, Capital Federal",Esmeralda 629,"Esmeralda entre Tucuman y Viamonte, al frente ...",15000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,30.0,1.0,NaN,NaN,NaN,8166.666667
9102,53346820,220000.0,"Centro / Microcentro, Capital Federal",Esmeralda 300,Departamento de 2 ambientes en zona de Microce...,59370.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,45.0,2.0,1.0,NaN,NaN,4888.888889
9103,53346612,220000.0,"Centro / Microcentro, Capital Federal",Esmeralda 300,Departamento de 2 ambientes en edificio con se...,50725.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,40.0,2.0,1.0,NaN,NaN,5500.000000
9104,53360616,400000.0,"Palermo, Capital Federal",Cerviño al 3200,"Excelente monoambiente, con amplio balcón terr...",23000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,55.0,1.0,NaN,NaN,NaN,7272.727273


In [35]:
df.value_counts("currency")

currency
ARS          5790
USD          2715
Consultar      29
Name: count, dtype: int64

In [36]:
df_ars = df.copy()
df_ars = df_ars[df_ars["currency"] == "ARS"].reset_index(drop = True)
df_ars["Expensas / alquiler"] = df_ars["Expensas"] / df_ars["Price"]
df_ars["Expensas / m2"] = df_ars["Expensas"] / df_ars["metros_cuadrados"]
df_ars["Precio / m2"] = df_ars["Price"] / df_ars["metros_cuadrados"]

In [37]:
df_usd = df.copy()
df_usd = df_usd[df_usd["currency"] == "USD"].reset_index(drop = True)

In [38]:
df_ars["Expensas"].isna().sum()

0

In [39]:
df_ars

,id,Price,Location,Address,Description,Expensas,Link,scrap_date,currency,metros_cuadrados,ambientes,dormitorios,baños,cocheras,Alquiler / m2,Expensas / alquiler,Expensas / m2,Precio / m2
0,53380145,550000.0,"Palermo, Capital Federal",Bulnes al 1900,"Semipiso 3 ambientes, con balcón al frente. Mu...",65000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,53.0,3.0,2.0,NaN,NaN,10377.358491,0.118182,1226.415094,10377.358491
1,53216300,900000.0,"Palermo, Capital Federal",Guise al 1700,Guise al 1700Magnífico 4 ambientes en Barrio N...,133000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,100.0,4.0,3.0,NaN,1.0,9000.000000,0.147778,1330.000000,9000.000000
2,53255302,550000.0,"Villa Urquiza, Capital Federal",Olazabal 5200 3°,Cómodo 3 ambientes ubicado en el corazón de Vi...,85000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,60.0,3.0,2.0,NaN,NaN,9166.666667,0.154545,1416.666667,9166.666667
3,53233935,350000.0,"Palermo, Capital Federal",Avenida Luis María Campos 100,Corredor Responsable: oscar N. Liberman cucicb...,33000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,34.0,1.0,NaN,NaN,NaN,10294.117647,0.094286,970.588235,10294.117647
4,53298077,570000.0,"Palermo, Capital Federal",Av. Las Heras al 4000,Edificio de buena categoría de unos 45 años de...,40000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,55.0,3.0,2.0,NaN,NaN,10363.636364,0.070175,727.272727,10363.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5785,45618429,245000.0,"Centro / Microcentro, Capital Federal",Esmeralda 629,"Esmeralda entre Tucuman y Viamonte, al frente ...",15000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,30.0,1.0,NaN,NaN,NaN,8166.666667,0.061224,500.000000,8166.666667
5786,53346820,220000.0,"Centro / Microcentro, Capital Federal",Esmeralda 300,Departamento de 2 ambientes en zona de Microce...,59370.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,45.0,2.0,1.0,NaN,NaN,4888.888889,0.269864,1319.333333,4888.888889
5787,53346612,220000.0,"Centro / Microcentro, Capital Federal",Esmeralda 300,Departamento de 2 ambientes en edificio con se...,50725.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,40.0,2.0,1.0,NaN,NaN,5500.000000,0.230568,1268.125000,5500.000000
5788,53360616,400000.0,"Palermo, Capital Federal",Cerviño al 3200,"Excelente monoambiente, con amplio balcón terr...",23000.0,https://www.zonaprop.com.ar/propiedades/clasif...,2024-04-09 12:47:43.337634,ARS,55.0,1.0,NaN,NaN,NaN,7272.727273,0.057500,418.181818,7272.727273


In [40]:
df_ars = df_ars[
                (df_ars["Expensas"]<=200_000)
                & (df_ars["Expensas"].notna())
                & (df_ars["Expensas"]>=10_000)
                & (df_ars["Price"]<=800_000)
                & (df_ars["Price"]>=10_000)
                & (df_ars["Expensas / alquiler"]<=1)
                & (df_ars["Expensas / m2"]<=10_000)
                & (df_ars["Expensas / m2"]>=10)
                ]
df_ars = df_ars.sort_values("Location")

In [41]:
df_ars.sort_values("Expensas / m2").iloc[-2,7]

Timestamp('2024-04-09 12:47:43.337634')

In [42]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Expensas"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})
fig.update_xaxes(type = "category")

In [51]:
fig = go.Figure()
fig.add_trace(go.Box(x = df_ars["Location"],y = df_ars["Precio / m2"]))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [45]:
tmp_df = df_ars.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Expensas / m2"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250})

In [46]:
df_usd = df_usd[
    (df_usd["metros_cuadrados"]<=60)
    & (df_usd["metros_cuadrados"].notna())
    & (df_usd["Price"] <= 2_000)
]

In [47]:
df_usd.sort_values("Price", ascending=True)[:20].iloc[18,5]

34000.0

In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_usd["Location"],y = df_usd["Price"], mode = "markers"))
fig.update_layout(template = None, height = 600, margin = {'b':250})

In [49]:
tmp_df = df_usd.copy()
tmp_df = tmp_df.groupby("Location", as_index=False).median(numeric_only = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x = tmp_df["Location"], y = tmp_df["Price"], mode = "markers"))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  )

In [50]:
fig = go.Figure()
fig.add_trace(go.Box(y = df_usd["Price"], x = df_usd["Location"]))
fig.update_layout(template = None, margin = {"b":250}, title_text = "Costo de alquiler en USD. Fuente Zonaprop",
                  height = 1_000)
fig.show()